In [ ]:

import numpy as np

# load data
from toy_data import N_CLASSES, N_WORKERS, votes

from peerannot.models import DawidSkene


/home/jozef/Desktop/repos/peerannot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [197]:
# dirty implementation just for comparison

class PoooledMultinomialBinaryCrude(DawidSkene):
    def _m_step(self) -> None:
        """Maximizing log likelihood with a single confusion matrix shared across all workers."""

        self.rho = self.T.sum(0) / self.n_task

        pi = np.zeros((self.n_workers, self.n_classes, self.n_classes))
        for q in range(self.n_classes):
            pij = self.T[:, q] @ self.crowd_matrix.transpose((1, 0, 2))
            denom = pij.sum(1)
            pi[:, q, :] = pij / np.where(denom <= 0, -1e9, denom).reshape(
                -1, 1,
            )


        alpha = np.diag(np.sum(pi,axis=0)/self.n_workers).mean()
        pij = np.full((self.n_classes, self.n_classes), (1 - alpha) / (self.n_classes - 1))
        np.fill_diagonal(pij, alpha)


        for k in range(self.n_workers):
            pi[k] = pij

        self.pi = pi

pmbc = PoooledMultinomialBinaryCrude(votes, N_WORKERS, N_CLASSES)
pmbc.run(maxiter=70)
pmbc.get_answers()

2025-03-27 16:28:05.276 | DEBUG    | peerannot.models.aggregation.DS:init_crowd_matrix:105 - Dense crowd matrix  5904
2025-03-27 16:28:05.277 | DEBUG    | peerannot.models.aggregation.DS:__init__:83 - Dense Crowd matrix5904
2025-03-27 16:28:05.277 | DEBUG    | peerannot.models.aggregation.DS:init_T:112 - Size of T before calc: 1568
2025-03-27 16:28:05.278 | DEBUG    | peerannot.models.aggregation.DS:init_T:116 - Size of T: 1568
Finished:  87%|████████▋ | 61/70 [00:00<00:00, 712.47it/s]


array([2, 2, 7, 1, 4, 0, 3, 4, 7, 1, 0, 3, 7, 3, 4, 7, 4, 7, 3, 7])

In [196]:
class PoooledMultinomialBinary(DawidSkene):
    def _m_step(self) -> None:
        """Maximizing log likelihood with a single confusion matrix shared across all workers."""

        self.rho = self.T.sum(0) / self.n_task

        diag_votes = np.einsum("tq, tiq -> q", self.T, self.crowd_matrix)
        denom = np.einsum("tq, tij -> q", self.T, self.crowd_matrix)
        

        self.alpha = (diag_votes/denom).mean()
        self.off_diag_alpha = (1 - self.alpha) / (self.n_classes - 1)
        # equivalent to:
        # sum_y_nj = np.sum(self.crowd_matrix, axis=1)
        # numerator = np.dot(self.T.T, sum_y_nj)
        # denom = np.sum(numerator,axis=1, keepdims=True)
        # alpha =  np.diag(numerator/denom).mean()


    def _e_step(self):
        T = np.zeros((self.n_task, self.n_classes))
        
        for i in range(self.n_task):
            for j in range(self.n_classes):
                worker_labels = self.crowd_matrix[i]

                diag_contrib = np.prod(np.power(self.alpha, worker_labels[:, j]))
                mask = np.ones(self.n_classes, dtype=bool)
                mask[j] = False
                off_diag_contrib = np.prod(np.power(self.off_diag_alpha, worker_labels[:, mask]))
                
                T[i,j] = diag_contrib * off_diag_contrib * self.rho[j]

        self.denom_e_step = T.sum(1, keepdims=True)
        self.T = np.where(self.denom_e_step > 0, T / self.denom_e_step, T)


pmb = PoooledMultinomialBinary(votes, N_WORKERS, N_CLASSES)

pmb.run(maxiter=100)
pmb.get_answers()



2025-03-27 16:27:57.420 | DEBUG    | peerannot.models.aggregation.DS:init_crowd_matrix:105 - Dense crowd matrix  5904
2025-03-27 16:27:57.421 | DEBUG    | peerannot.models.aggregation.DS:__init__:83 - Dense Crowd matrix5904
2025-03-27 16:27:57.422 | DEBUG    | peerannot.models.aggregation.DS:init_T:112 - Size of T before calc: 1568
2025-03-27 16:27:57.422 | DEBUG    | peerannot.models.aggregation.DS:init_T:116 - Size of T: 1568
Finished:  51%|█████     | 51/100 [00:00<00:00, 230.04it/s]       


array([2, 2, 7, 1, 4, 0, 3, 4, 7, 1, 0, 3, 7, 3, 4, 7, 4, 7, 3, 7])

In [43]:
class PoooledMultinomialBinary(DawidSkene):
    def _m_step(self) -> None:
        """Maximizing log likelihood with a single confusion matrix shared across all workers."""

        self.rho = self.T.sum(0) / self.n_task

        diag_votes = np.einsum("tq, tiq -> q", self.T, self.crowd_matrix)

        denom = np.einsum("tq, tij -> q", self.T, self.crowd_matrix)

        self.alpha = (diag_votes/np.where(denom > 0, denom, 1e-9)).mean()
        self.off_diag_alpha = (1 - self.alpha) / (self.n_classes - 1)
        
    def _e_step(self):
        """Expectation step: estimate probability of true labels given current parameters."""

        T = np.zeros((self.n_task, self.n_classes))

        for i in range(self.n_task):
            worker_labels = self.crowd_matrix[i].sum(0)
            for j in range(self.n_classes):
                n_ij = worker_labels[j]  # Number of annotators labeling task i as class j
                n_i = worker_labels.sum()  # Total number of annotations for task i

                diag_contrib = self.alpha ** n_ij
                off_diag_contrib = self.off_diag_alpha ** (n_i - n_ij)

                T[i, j] = diag_contrib * off_diag_contrib * self.rho[j]
        self.denom_e_step = T.sum(1, keepdims=True)
        self.T = np.where(self.denom_e_step > 0, T / self.denom_e_step, T)

pmb = PoooledMultinomialBinary(votes, N_WORKERS, N_CLASSES)
pmb.run(maxiter=100)
pmb.get_answers()


2025-03-27 15:19:02.051 | DEBUG    | peerannot.models.aggregation.DS:init_crowd_matrix:105 - Dense crowd matrix  5904
2025-03-27 15:19:02.052 | DEBUG    | peerannot.models.aggregation.DS:__init__:83 - Dense Crowd matrix5904
2025-03-27 15:19:02.053 | DEBUG    | peerannot.models.aggregation.DS:init_T:112 - Size of T before calc: 1568
2025-03-27 15:19:02.053 | DEBUG    | peerannot.models.aggregation.DS:init_T:116 - Size of T: 1568
Finished:  51%|█████     | 51/100 [00:00<00:00, 2348.90it/s]

0.5611882716049382
0.5083423727791125
0.4857513922405727
0.47450574356963304
0.46852854496738505
0.4652348418003532
0.46337661016830217
0.4623093183490329
0.461686761844257
0.46131824483927375
0.4610968282507372
0.4609616791351889
0.46087775732595304
0.4608246436685348
0.4607903051851706
0.4607675721839727
0.4607521252123096
0.46074133245332877
0.46073357136169235
0.4607278289080042
0.46072346375104534
0.460720063449222
0.46071735797961616
0.4607151669164742
0.4607133669001838
0.4607118714342609
0.4607106182280615
0.4607095611906515
0.46070866531135746
0.4607079033443758
0.4607072536291802
0.4607066986318864
0.46070622394863237
0.46070581760842294
0.4607054695727811
0.46070517136691447
0.46070491580053463
0.46070469675120296
0.4607045089924127
0.46070434805454474
0.4607042101106432
0.46070409188141576
0.4607039905554686
0.46070390372186143
0.46070382931278125
0.4607037655546413
0.46070371092625845
0.46070366412302427
0.4607036240261784
0.4607035896764365
0.460703560251351


array([2, 2, 7, 1, 4, 0, 3, 4, 7, 1, 0, 3, 7, 3, 4, 7, 4, 7, 3, 7])